In [26]:
%pip install "zenml"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2  # required for Colab

import IPython

# automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

Note: you may need to restart the kernel to use updated packages.
⠸ Installing integrations.....
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 2.4.7
    Uninstalling pyparsing-2.4.7:
      Successfully uninstalled pyparsing-2.4.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-lsp 4.0.1 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2.86.0 which is incompatible.
beatrix-jupyterlab 2023.46.184821 requires jupyter-server~=1.16, but you have jupyter-server 2.5.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
Note: you may need to restart

{'status': 'ok', 'restart': True}

In [11]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("zenml_password")
secret_value_1 = user_secrets.get_secret("zenml_user")

In [16]:
! zenml connect --url 'https://bsenst-zenml-server.hf.space/' --username=secret_value_1 --password=secret_value_0

Connecting to: 'https://bsenst-zenml-server.hf.space/'...
Updated the global store configuration.


In [1]:
! zenml status

Using configuration from: '/root/.config/zenml'
Local store files are located at: '/root/.config/zenml/local_stores'
Active repository root: /kaggle/working
Connected to a ZenML server: 'https://bsenst-zenml-server.hf.space'
The current user is: 'default'
The active workspace is: 'default' (repository)
The active stack is: 'default' (repository)


In [18]:
# https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
! wget --directory-prefix=input https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet
! wget --directory-prefix=input https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet
! wget --directory-prefix=input https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-03.parquet

--2023-05-18 21:22:25--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.222.137.46, 52.222.137.56, 52.222.137.206, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.222.137.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1254291 (1.2M) [binary/octet-stream]
Saving to: ‘input/green_tripdata_2022-01.parquet’

green_tripdata_2022 100%[===================>]   1.20M  --.-KB/s    in 0.02s   

2023-05-18 21:22:25 (48.4 MB/s) - ‘input/green_tripdata_2022-01.parquet’ saved [1254291/1254291]

--2023-05-18 21:22:25--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.222.137.46, 52.222.137.90, 52.222.137.206, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.222.137.46|:443... connected.
HT

In [19]:
! wget https://github.com/DataTalksClub/mlops-zoomcamp/raw/main/cohorts/2023/02-experiment-tracking/homework/preprocess_data.py

--2023-05-18 21:23:04--  https://github.com/DataTalksClub/mlops-zoomcamp/raw/main/cohorts/2023/02-experiment-tracking/homework/preprocess_data.py
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2023/02-experiment-tracking/homework/preprocess_data.py [following]
--2023-05-18 21:23:04--  https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2023/02-experiment-tracking/homework/preprocess_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2512 (2.5K) [text/plain]
Saving to: ‘preprocess_data.py’

preprocess_data.py  100%[==========

In [20]:
! python preprocess_data.py --raw_data_path input --dest_path ./output

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [21]:
! rm -rf .zen
! zenml init

⠋ Initializing ZenML repository at /kaggle/working.
⠙ Initializing ZenML repository at /kaggle/working.
⠹ Initializing ZenML repository at /kaggle/working.
⠸ Initializing ZenML repository at /kaggle/working.
⠼ Initializing ZenML repository at /kaggle/working.
⠴ Initializing ZenML repository at /kaggle/working.
⠦ Initializing ZenML repository at /kaggle/working.
⠧ Initializing ZenML repository at /kaggle/working.
⠇ Initializing ZenML repository at /kaggle/working.
⠏ Initializing ZenML repository at /kaggle/working.
⠋ Initializing ZenML repository at /kaggle/working.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
⠙ Initializing ZenML repository at /kaggle/working.
⠹ Initializing ZenML repository at /kaggle/working.
⠸ Initializing ZenML repository at /kaggle/working.
⠼ Initializing ZenML repository at /kaggle/working.
⠴ Initializing ZenML repository at /kaggle/working.
⠦ Initializing ZenML repository at /kaggle/working.
⠧ Initializing ZenML repos

In [44]:
import os
import pickle
import scipy
import numpy

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from zenml.steps import step, Output
from zenml.pipelines import pipeline

def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

@step
def importer() -> Output(
    X_train=scipy.sparse.csr.csr_matrix,
    X_test=scipy.sparse.csr.csr_matrix,
    y_train=numpy.ndarray,
    y_test=numpy.ndarray,
):
    X_train, y_train = load_pickle(os.path.join("./output", "train.pkl"))
    X_val, y_val = load_pickle(os.path.join("./output", "val.pkl"))
    return X_train, X_val, y_train, y_val

@step
def trainer(X_train: scipy.sparse.csr.csr_matrix, y_train: numpy.ndarray) -> RandomForestRegressor:
    rf = RandomForestRegressor(max_depth=10, random_state=0)
    rf.fit(X_train, y_train)
    return rf

@step
def evaluator(X_test: scipy.sparse.csr.csr_matrix, y_test: numpy.ndarray, model: RandomForestRegressor) -> numpy.float64:
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print(f"RMSE: {rmse}")   
    return rmse

@pipeline
def pipeline(importer, trainer, evaluator):
    X_train, X_test, y_train, y_test = importer()
    model = trainer(X_train=X_train, y_train=y_train)
    evaluator(X_test=X_test, y_test=y_test, model=model)

pipeline(
    importer=importer(), trainer=trainer(), evaluator=evaluator()
).run(unlisted=True)

Running unlisted pipeline on stack default (caching enabled)
Step importer has started.
Using cached version of importer.
Step trainer has started.
Using cached version of trainer.
Step evaluator has started.
RMSE: 2.453983836538874
Step evaluator has finished in 0.957s.
Pipeline run pipeline-2023_05_18-22_40_13_373840 has finished in 4.305s.
Dashboard URL: https://bsenst-zenml-server.hf.space/pipelines/all-runs
